## OSWorld

In [ ]:
from agential.agents.OSWorldBaseline.agent import OSWorldBaselineAgent

In [8]:
"""
To get accessibility_tree, retreive obs by running this line
- obs, reward, done, info = env.step("pyautogui.rightClick()")
- obs["accessibility_tree"]
"""
with open("accessibility_tree.txt", "r", encoding="utf-8") as file:
    accessibility_tree = file.read()

instruction = "Please help me to find the nearest restaurant."

"""
Image of screen is to be provided by user
"""
obs = {"screenshot": open("output_image.jpeg", 'rb').read()}


In [4]:
agent = OSWorldBaselineAgent(
    model="gpt-4o",
    observation_type="screenshot",
)

In [5]:
response, actions, messages = agent.generate(instruction, obs)

In [ ]:
print(f"Response: {response}")
print(f"Actions: {actions}")
print(f"Messages: {messages}")